# Imports

In [1]:
import aurorax
import datetime
import pprint
import os
import pandas as pd

# Upload ephemeris data

In [5]:
# read API key from environment vars
api_key = os.environ["AURORAX_API_KEY"]
aurorax.authenticate(api_key)

In [6]:
# set values
program = "notebook-program"
platform = "notebook-platform"
instrument_type = "notebook-instrument-type"
metadata = {
    "dp_meta1": "testing1",
    "dp_meta2": "testing2"
}
start_dt = datetime.datetime(2020, 1, 1, 0, 0, 0)
end_dt = start_dt.replace(hour=23, minute=59, second=59)
data_product_type = "keogram"
url = "notebook_testing.jpg"

In [8]:
# get the ephemeris source
source = aurorax.sources.get(program,
                            platform,
                            instrument_type)
identifier = source.identifier
print("Identified: %d" % (identifier))

Identified: 130


In [9]:
# create data product object
dp = aurorax.data_products.DataProduct(data_source=source,
                                      data_product_type=data_product_type,
                                      start=start_dt,
                                      end=end_dt,
                                      url=url,
                                      metadata=metadata)
pprint.pprint(dp)

{'data_product_type': 'keogram',
 'data_source': {'data_product_metadata_schema': [],
 'display_name': 'Test Instrument',
 'ephemeris_metadata_schema': [],
 'identifier': 130,
 'instrument_type': 'notebook-instrument-type',
 'maintainers': [],
 'metadata': {},
 'owner': None,
 'platform': 'notebook-platform',
 'program': 'notebook-program',
 'source_type': 'ground'},
 'end': datetime.datetime(2020, 1, 1, 23, 59, 59),
 'metadata': {'dp_meta1': 'testing1', 'dp_meta2': 'testing2'},
 'start': datetime.datetime(2020, 1, 1, 0, 0),
 'url': 'notebook_testing.jpg'}


In [10]:
# set records array
#
# we do this because you can upload multiple ephemeris records, so the 
# standard object being passed to the upload function is a list
records = []
records.append(dp)

In [11]:
# upload record
try:
    aurorax.data_products.upload(identifier, records)
except aurorax.AuroraXException as e:
    print("Error uploading: %s" % (str(e)))

# Check that the data was uploaded by retrieving it

In [12]:
# perform search
s = aurorax.data_products.search(start_dt,
                                end_dt,
                                programs=[program],
                                platforms=[platform],
                                instrument_types=[instrument_type])

In [13]:
# show data as a pandas dataframe
data_products = [dp.__dict__ for dp in s.data]
df = pd.DataFrame(data_products)
df

,data_source,data_product_type,start,end,url,metadata
0,"{'data_product_metadata_schema': [],\n 'displa...",keogram,2020-01-01,2020-01-01 23:59:00,notebook_testing.jpg,"{'dp_meta1': 'testing1', 'dp_meta2': 'testing2'}"
